In [1]:
import cv2
import numpy as np

In [2]:
# Parameters for binarize threshold and Canny Line detection
bi_thd = 20
bi_max = 255

#Parameters for canny
canny_thrd1=10
canny_thrd2=120

#Parameters for visualization
nor_color = (255, 255, 255)
warn_color = (0,0,255)
font = cv2.FONT_HERSHEY_SIMPLEX
color = (0, 255, 255)

minContourSize = 200

In [3]:
def displayResizedImg(name, img, factor):
    imgDis = cv2.resize(img, None, fx=factor, fy=factor, interpolation=cv2.INTER_AREA)
    cv2.imshow(name, imgDis)
    return imgDis

In [4]:
def joinContours(src, contours):
    # create an empty mask
    mask = np.zeros(src.shape[:2],dtype=np.uint8)
    # draw filled boundingrects if the contour is large enough
    for c in contours:
            if cv2.contourArea(c) > minContourSize:
                    x,y,w,h  = cv2.boundingRect(c)
                    cv2.rectangle(mask,(x,y),(x+w,y+h),(255),-1)

    # find the contours on the mask (with solid drawn shapes) and draw outline on input image
    contours,hierarchy=cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    return contours
    #for c in contours:
    #                cv2.drawContours(res,[c],0,(0,0,255),2)
    ## show image
    #cv2.imshow("Contour",res)


In [5]:
def get_contour_precedence(contour, cols):
    tolerance_factor = 10
    origin = cv2.boundingRect(contour)
    return ((origin[1] // tolerance_factor) * tolerance_factor) * cols + origin[0]

In [6]:
def scale_contour(cnt, scale):
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    cnt_norm = cnt - [cx, cy]
    cnt_scaled = cnt_norm * scale
    cnt_scaled = cnt_scaled + [cx, cy]
    cnt_scaled = cnt_scaled.astype(np.int32)

    return cnt_scaled

In [7]:
#read from file
#imRgb = cv2.imread("D:/Research/opencv/image/X-Rite-Color-Checker-CC-Classic-array-with-CIELab-values.png")
imRgb = cv2.imread("D:/Research/opencv/image/colorchecker.jpg")

imGray = cv2.cvtColor(imRgb, cv2.COLOR_BGR2GRAY)
imLab = cv2.cvtColor(imRgb, cv2.COLOR_BGR2LAB)
L = imLab[:, :, 0]
a = imLab[:, :, 1]
b = imLab[:, :, 2]

In [8]:
#detect ROI

#find gradient
imgTemp = cv2.blur(imGray, (3,3))
canny = cv2.Canny(imgTemp, canny_thrd1, canny_thrd2)

contours, hierarchy = cv2.findContours(canny,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
contours = joinContours(imRgb, contours)
#sort
contours.sort(key=lambda x:get_contour_precedence(x, imRgb.shape[1]))

cnt_scaled = [np.array([[[10,10]], [[10,50]], [[50,50]], [[50,10]]])]
#cnt_scaled = [np.array(contours)]
for c in contours:
    np.append(cnt_scaled, np.array([[[0,0]], [[0,50]], [[50,50]], [[50,0]]]))
    #np.append(cnt_scaled, scale_contour(c, 0.5))

#print(cnt_scaled)

cv2.drawContours(imRgb, cnt_scaled, -1, color, 3)
i=0
for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    print(i,x,y,w,h)
    cv2.putText(imRgb,'{}'.format(i), (x, y), font, 1, nor_color, 1, cv2.LINE_AA)
    i+=1
    


0 8 30 74 74
1 90 30 74 74
2 172 30 75 74
3 255 30 74 74
4 337 30 74 74
5 419 30 74 74
6 8 112 74 74
7 90 112 74 74
8 172 112 75 74
9 254 112 75 74
10 337 112 74 74
11 419 112 74 74
12 8 194 74 74
13 90 194 74 74
14 172 194 75 74
15 254 194 75 74
16 337 194 74 74
17 419 194 74 74
18 8 276 74 74
19 90 276 74 74
20 172 276 75 74
21 254 276 75 74
22 337 276 74 74
23 417 275 78 77


In [9]:
displayResizedImg('ori', imRgb,1)
#displayResizedImg('L',L,0.5)
#displayResizedImg('a',a,0.5)
#displayResizedImg('b',b,0.5)
cv2.waitKey()
cv2.destroyAllWindows()